In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os, shutil
import numpy as np
from matplotlib import pyplot as plt, pylab as pylab
import mdtraj as md
import nglview as nv

In [3]:
%matplotlib inline
plt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = 12, 8

# Barnase-Barstar


From PDB:1BRS.



En el paper de Nuria Plattner y Frank Noe toman como wildtype la cadena B de Barnase y la F superimpuesta a la E para Barstar.

In [9]:
BB_pdb = md.load('1brs.pdb')
view = nv.show_mdtraj(BB_pdb)
view

NGLWidget()

In [13]:
top=BB_pdb.topology

In [18]:
chain=top.chain(0)

In [19]:
chain.index

0

In [ ]:
top.select